In [1]:
from langchain_community.chat_models import ChatOllama
import os

from langchain_unstructured import UnstructuredLoader
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
llm = ChatOllama(model = "llama3.1")

/tmp/ipykernel_1685442/3072841417.py:7: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model = "llama3.1")


In [2]:
import torch

In [3]:
file_path = "test.pdf"
print("文件路径:", os.path.abspath(file_path))


文件路径: /home/wangao/RAG/test.pdf


In [11]:

from langchain.schema import Document
import pdfplumber

loader = UnstructuredLoader(file_path)
docs = loader.load()  
page_docs = []
with pdfplumber.open(file_path) as pdf:
    for page_number, page in enumerate(pdf.pages):
        page_content = page.extract_text()  # 提取每页文本
        if page_content:  # 检查是否成功提取文本
            # 如果该页内容超出500字符，则进一步分割
            if len(page_content) > 500:
                split_texts = text_splitter.split_text(page_content)
                for text in split_texts:
                    page_docs.append({
                        "page_number": page_number + 1,
                        "content": text
                    })
            else:
                # 如果内容不超过500字符，直接添加为一个文档
                page_docs.append({
                    "page_number": page_number + 1,
                    "content": page_content
                })



分割后的文档数量: 251
前两个分割后的文档内容示例:
文档 1 (页码 1):
상표법
상표법
[시행 2024. 8. 7.] [법률 제20200호, 2024. 2. 6., 타법개정]
제1장 총칙
제1조(목적)
제2조(정의)
제3조(상표등록을 받을 수 있는 자)
제4조(미성년자 등의 행위능력)
제5조(법인이 아닌 사단 등)
제6조(재외자의 상표관리인)
제7조(대리권의 범위)
제8조(대리권의 증명)
제9조(행위능력 등의 흠에 대한 추인)
제10조(대리권의 불소멸)
제11조(개별대리)
제12조(대리인의 선임 또는 교체 명령 등)
제13조(복수당사자의 대표)
제14조(「민사소송법」의 준용)
제15조(재외자의 재판관할)
제16조(기간의 계산)
제17조(기간의 연장 등)
제18조(절차의 무효)
제19조(절차의 추후 보완)
제20조(절차의 효력 승계)
제21조(절차의 속행)
제22조(절차의 중단)
제23조(중단된 절차의 수계)
제24조(수계신청)
제25조(절차의 중지)
제26조(중단 또는 중지의 효과)
제27조(외국인의 권리능력)
제28조(서류 제출의 효력 발생 시기)
文档 2 (页码 1):
제27조(외국인의 권리능력)
제28조(서류 제출의 효력 발생 시기)
제29조(고유번호의 기재)
제30조(전자문서에 의한 상표에 관한 절차의 수행)
제31조(전자문서 이용신고 및 전자서명)
제32조(정보통신망을 이용한 통지 등의 수행)
제2장 상표등록요건 및 상표등록출원
제33조(상표등록의 요건)
제34조(상표등록을 받을 수 없는 상표)
제35조(선출원)
제36조(상표등록출원)
제37조(상표등록출원일의 인정 등)
제38조(1상표 1출원)
제39조(절차의 보정)
제40조(출원공고결정 전의 보정)
법제처 1 국가법령정보센터
文档 1 长度: 497
文档 2 长度: 292
文档 3 长度: 491
文档 4 长度: 401
文档 5 长度: 464


In [12]:
documents = [
    Document(page_content=doc["content"], metadata={"page_number": doc["page_number"]})
    for doc in page_docs
]

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}  
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vector_store = Chroma.from_documents(documents, embeddings)




INFO: PyTorch version 2.5.1+cu121 available.
INFO: Load pretrained SentenceTransformer: BAAI/bge-m3
INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [13]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.1,
    },
)

In [14]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("""Anser the quetion using the provided context. if the answer is not contained in context, say "can not find in context"\n\n
                                      Context:{context}
                                      Question{question}
                                      Answer:
                                      
                                      """)

In [15]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
chain = (
     {
          "context":retriever, "question":RunnablePassthrough()
     }
     | prompt
     | llm
     |StrOutputParser()
)

In [22]:
reponse = chain.invoke("what's the hight montain in the world?")

print(reponse)

/home/wangao/.v12/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:1083: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page_number': 5}, page_content='제185조(보정의 특례)\n제186조(출원 변경의 특례)\n제187조(출원 분할의 특례)\n제188조(파리협약에 따른 우선권 주장의 특례)\n제189조(출원 시 및 우선심사의 특례)\n제190조(거절이유 통지의 특례)\n제191조(출원공고의 특례)\n제192조(손실보상청구권의 특례)\n제193조(상표등록결정 및 직권에 의한 보정 등의 특례)\n제193조의2(재심사 청구의 특례)\n제193조의3(상표등록여부결정의 방식에 관한 특례)\n제194조(상표등록료 등의 특례)\n제195조(상표등록료 등의 반환의 특례)\n제196조(상표원부에의 등록의 특례)\n제197조(상표권 설정등록의 특례)\n제198조(상표권 존속기간 등의 특례)\n제199조(지정상품추가등록출원의 특례)\n제200조\n제201조(상표권등록 효력의 특례)\n제202조(국제등록 소멸의 효과)\n제203조(상표권 포기의 특례)\n제204조(존속기간갱신등록의 무효심판 등의 특례)\n제3절 상표등록출원의 특례\n제205조(국제등록 소멸 후의 상표등록출원의 특례)'), -0.06226546094590413), (Document(metadata={'page_number': 44}, page_content='제9장 「표장의 국제등록에 관한 마드리드협정에 대한 의정서」에 따른 국제출원\n제1절 국제출원 등\n제167조(국제출원) 「표장의 국제등록에 관한 마드리드협정에 대한 의정서」(이하 “마드리드 의정서”라 한다) 제2조(1)에\n따른 국제등록(이하 “국제등록”이라 한다)을 받으려는 자는 다음 각 호의 어느 하나에 해당하는 상표등록출원 또는\n법제처 44 국가법

Can't find in context


In [21]:
reponse2 = chain.invoke("""is "심판청구서 등의 각하에 관한 적용례" in the list?""")
print(reponse2)

Yes

The context contains a series of documents with metadata and page content related to patent law, specifically regarding trademark registration and dispute resolution. In one of the documents (Document(metadata={'page_number': 38}), page_content='...'), there is a section on "심판청구서 등의 각하에 관한 적용례" which translates to "Application examples for dismissal of petitions, etc.".

This suggests that the context indeed contains information related to "심판청구서 등의 각하에 관한 적용례".
